In [1]:
# Dependencies
import googlemaps
import pandas as pd

In [ ]:
gmaps = googlemaps.Client(key="SECRET")

In [ ]:
bigfoot_df = pd.read_json('../data/filtered_years_clean.json')

In [ ]:
def geocode_with_fallback(row):
    try:
        # Try nearest town, state
        location = gmaps.geocode(f"{row['nearest town']}, {row['state']}")
        if location:
            return location[0]['geometry']['location']['lat'], location[0]['geometry']['location']['lng']

        # Fallback to county, state
        location = gmaps.geocode(f"{row['county']}, {row['state']}")
        if location:
            return location[0]['geometry']['location']['lat'], location[0]['geometry']['location']['lng']

        # If neither works, return None
        return None, None
    except Exception as e:
        print(f"Error geocoding {row['nearest town']}, {row['state']} or {row['county']}, {row['state']}: {e}")
        return None, None

In [ ]:
bigfoot_stub = bigfoot_df.iloc[:10]

In [ ]:
bigfoot_df[['latitude', 'Longitude']] = bigfoot_df.apply(
    lambda row: pd.Series(geocode_with_fallback(row)), axis=1
)

In [ ]:
bigfoot_df = bigfoot_df.drop(columns='Unnamed: 0')
bigfoot_df.to_json('../data/bigfoot_coords_df.json', orient='records')

In [8]:
bigfoot_df = pd.read_json('../data/bigfoot_coords_df.json')

bigfoot_df.columns

Index(['report_number', 'report_class', 'year', 'season', 'month', 'state',
       'county', 'location_details', 'nearest_town', 'nearest_road',
       'observed', 'also_noticed', 'other_witnesses', 'other_stories',
       'time_and_conditions', 'environment', 'date', 'a_&_g_references',
       'latitude', 'longitude'],
      dtype='object')

In [10]:
# drop longitudes that would be outside of the USA / Canada
filtered_coords = bigfoot_df[(bigfoot_df['longitude'] >= -125) & (bigfoot_df['longitude'] <= -67)]

In [11]:
final_columns = ['report_number', 'report_class', 'state', 'county', 'latitude', 'longitude', 'season', 'month', 'observed']

clean_coords_list = filtered_coords[final_columns]
clean_coords_list.to_json('../data/bigfoot_coordinates_clean_cols.json', orient='records')